In [1]:
import torch
import torch.nn as nn
import math

class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps: float = 10**-6) -> None:
        super().__init__()
        self.eps = eps  # Küçük bir değer, sıfıra bölünmeyi önlemek için
        self.alpha = nn.Parameter(torch.ones(features))  # Ölçeklendirme parametresi (öğrenilebilir)
        self.bias = nn.Parameter(torch.zeros(features))  # Kaydırma parametresi (öğrenilebilir)

    def forward(self, x):
        # Girdinin ortalamasını ve standart sapmasını hesapla
        mean = x.mean(dim=-1, keepdim=True)  # (batch, seq_len, 1)
        std = x.std(dim=-1, keepdim=True)  # (batch, seq_len, 1)
        # Normalizasyon formülü: (x - mean) / (std + eps) * alpha + bias
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)  # İlk lineer dönüşüm (w1 ve b1)
        self.dropout = nn.Dropout(dropout)  # Dropout katmanı
        self.linear_2 = nn.Linear(d_ff, d_model)  # İkinci lineer dönüşüm (w2 ve b2)

    def forward(self, x):
        # İleri beslemeli sinir ağı: Lineer -> ReLU -> Dropout -> Lineer
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))


class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model  # Gömme vektörlerinin boyutu
        self.vocab_size = vocab_size  # Kelime dağarcığı boyutu
        self.embedding = nn.Embedding(vocab_size, d_model)  # Gömme katmanı

    def forward(self, x):
        # Token indekslerini gömme vektörlerine dönüştür ve ölçeklendir
        return self.embedding(x) * math.sqrt(self.d_model)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model  # Gömme vektörlerinin boyutu
        self.seq_len = seq_len  # Maksimum dizi uzunluğu
        self.dropout = nn.Dropout(dropout)  # Dropout katmanı
        # Konumsal kodlama matrisini oluştur
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)  # Pozisyon vektörü
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # Bölme terimi
        pe[:, 0::2] = torch.sin(position * div_term)  # Çift indeksler için sinüs
        pe[:, 1::2] = torch.cos(position * div_term)  # Tek indeksler için kosinüs
        pe = pe.unsqueeze(0)  # Batch boyutu ekle
        self.register_buffer('pe', pe)  # Konumsal kodlamayı sabit olarak kaydet

    def forward(self, x):
        # Girdiye konumsal kodlamayı ekle
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)


class ResidualConnection(nn.Module):
    def __init__(self, features: int, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)  # Dropout katmanı
        self.norm = LayerNormalization(features)  # Katman normalizasyonu

    def forward(self, x, sublayer):
        # Artık bağlantı: x + dropout(sublayer(norm(x)))
        return x + self.dropout(sublayer(self.norm(x)))


class MultiHeadAttentionBlock(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model  # Gömme vektörlerinin boyutu
        self.h = h  # Dikkat kafalarının sayısı
        assert d_model % h == 0, "d_model is not divisible by h"  # d_model h'a bölünebilir olmalı
        self.d_k = d_model // h  # Her bir kafanın vektör boyutu
        self.w_q = nn.Linear(d_model, d_model, bias=False)  # Query dönüşümü
        self.w_k = nn.Linear(d_model, d_model, bias=False)  # Key dönüşümü
        self.w_v = nn.Linear(d_model, d_model, bias=False)  # Value dönüşümü
        self.w_o = nn.Linear(d_model, d_model, bias=False)  # Çıktı dönüşümü
        self.dropout = nn.Dropout(dropout)  # Dropout katmanı

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Dikkat skorlarını hesapla: (Q * K^T) / sqrt(d_k)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)  # Maskeli yerleri -∞ yap
        attention_scores = attention_scores.softmax(dim=-1)  # Softmax uygula
        if dropout is not None:
            attention_scores = dropout(attention_scores)  # Dropout uygula
        return (attention_scores @ value), attention_scores  # Çıktı ve dikkat skorları

    def forward(self, q, k, v, mask):
        query = self.w_q(q)  # Query dönüşümü
        key = self.w_k(k)  # Key dönüşümü
        value = self.w_v(v)  # Value dönüşümü
        # Çok kafalı dikkat için şekil değiştir
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
        # Dikkat mekanizmasını uygula
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        # Kafaları birleştir ve çıktı dönüşümü uygula
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        return self.w_o(x)


class EncoderBlock(nn.Module):
    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block  # Self-attention katmanı
        self.feed_forward_block = feed_forward_block  # İleri beslemeli sinir ağı
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])  # Artık bağlantılar

    def forward(self, x, src_mask):
        # Self-attention ve artık bağlantı
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        # İleri beslemeli sinir ağı ve artık bağlantı
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x


class Encoder(nn.Module):
    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers  # Encoder blokları
        self.norm = LayerNormalization(features)  # Son katman normalizasyonu

    def forward(self, x, mask):
        # Tüm encoder bloklarını uygula
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)  # Son katman normalizasyonu


class DecoderBlock(nn.Module):
    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block  # Self-attention katmanı
        self.cross_attention_block = cross_attention_block  # Cross-attention katmanı
        self.feed_forward_block = feed_forward_block  # İleri beslemeli sinir ağı
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])  # Artık bağlantılar

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # Self-attention ve artık bağlantı
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        # Cross-attention ve artık bağlantı
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        # İleri beslemeli sinir ağı ve artık bağlantı
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x


class Decoder(nn.Module):
    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers  # Decoder blokları
        self.norm = LayerNormalization(features)  # Son katman normalizasyonu

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # Tüm decoder bloklarını uygula
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)  # Son katman normalizasyonu


class ProjectionLayer(nn.Module):
    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)  # Lineer projeksiyon katmanı

    def forward(self, x) -> None:
        # Girdiyi kelime dağarcığı boyutuna projelendir
        return self.proj(x)


class Transformer(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder  # Encoder katmanı
        self.decoder = decoder  # Decoder katmanı
        self.src_embed = src_embed  # Kaynak gömme katmanı
        self.tgt_embed = tgt_embed  # Hedef gömme katmanı
        self.src_pos = src_pos  # Kaynak konumsal kodlama
        self.tgt_pos = tgt_pos  # Hedef konumsal kodlama
        self.projection_layer = projection_layer  # Projeksiyon katmanı

    def encode(self, src, src_mask):
        # Kaynak diziyi kodla
        src = self.src_embed(src)  # Gömme katmanı
        src = self.src_pos(src)  # Konumsal kodlama
        return self.encoder(src, src_mask)  # Encoder katmanı

    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # Hedef diziyi çöz
        tgt = self.tgt_embed(tgt)  # Gömme katmanı
        tgt = self.tgt_pos(tgt)  # Konumsal kodlama
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)  # Decoder katmanı

    def project(self, x):
        # Çıktıyı kelime dağarcığı boyutuna projelendir
        return self.projection_layer(x)


def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048) -> Transformer:
    # Gömme katmanlarını oluştur
    src_embed = InputEmbeddings(d_model, src_vocab_size)  # Kaynak gömme
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)  # Hedef gömme

    # Konumsal kodlama katmanlarını oluştur
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)  # Kaynak konumsal kodlama
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)  # Hedef konumsal kodlama

    # Encoder bloklarını oluştur
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)  # Self-attention
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)  # İleri beslemeli sinir ağı
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)  # Encoder bloğu
        encoder_blocks.append(encoder_block)

    # Decoder bloklarını oluştur
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)  # Self-attention
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)  # Cross-attention
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)  # İleri beslemeli sinir ağı
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)  # Decoder bloğu
        decoder_blocks.append(decoder_block)

    # Encoder ve Decoder'ı oluştur
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))  # Encoder
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))  # Decoder

    # Projeksiyon katmanını oluştur
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)  # Projeksiyon katmanı

    # Transformer modelini oluştur
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Parametreleri Xavier uniform ile başlat
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
import math
from torch.cuda.amp import GradScaler, autocast

# Yapılandırma parametreleri
CONFIG = {
    "batch_size": 64,
    "d_model": 512,
    "seq_len": 256,
    "num_epochs": 1,
    "lr": 1e-4,
    "hf_dataset": "wikitext",
    "dataset_name": "wikitext-103-raw-v1",
    "vocab_size": 30000,
    "dropout": 0.1,
    "n_layers": 6,
    "n_heads": 8,
    "d_ff": 1024
}

# Maskeleri oluşturma fonksiyonları
def create_padding_mask(seq):
    print("Padding maskesi oluşturuluyor...")
    mask = (seq != 0).unsqueeze(1).unsqueeze(2)
    print(f"Padding mask shape: {mask.shape}")
    return mask

def create_causal_mask(size):
    print("Causal maskesi oluşturuluyor...")
    mask = torch.tril(torch.ones(size, size)).bool()
    mask = mask.unsqueeze(0).unsqueeze(0)
    print(f"Causal mask shape: {mask.shape}")
    return mask

def create_mask(src, tgt):
    print("Maskeler birleştiriliyor...")
    src_padding_mask = create_padding_mask(src)
    tgt_padding_mask = create_padding_mask(tgt)
    causal_mask = create_causal_mask(tgt.size(1))
    tgt_mask = tgt_padding_mask & causal_mask
    print(f"Source mask shape: {src_padding_mask.shape}, Target mask shape: {tgt_mask.shape}")
    return src_padding_mask.to(src.device), tgt_mask.to(tgt.device)

# Veri Seti Sınıfı
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, seq_len):
        self.texts = data['text']
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        print(f"TextDataset oluşturuldu. Toplam örnek sayısı: {len(self.texts)}")
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        print(f"Tokenizing text: {text[:50]}...")  # İlk 50 karakteri göster
        tokens = self.tokenizer.encode(text).ids
        
        if len(tokens) < self.seq_len:
            tokens += [0] * (self.seq_len - len(tokens))
        else:
            tokens = tokens[:self.seq_len]
        
        print(f"Tokenized sequence length: {len(tokens)}")
        return torch.tensor(tokens[:-1]), torch.tensor(tokens[1:])

# Tokenizer Oluşturma
def build_tokenizer(dataset):
    print("Tokenizer oluşturuluyor...")
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(
        special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"], 
        vocab_size=CONFIG['vocab_size']
    )
    
    def batch_iterator():
        for i in range(0, len(dataset), 1000):
            batch = dataset[i:i+1000]['text']
            print(f"Tokenizer eğitimi için batch {i//1000 + 1} işleniyor...")
            yield batch
            
    tokenizer.train_from_iterator(batch_iterator(), trainer)
    print("Tokenizer eğitimi tamamlandı.")
    return tokenizer

# Veri Yükleme
def get_data():
    print("Veri seti yükleniyor...")
    dataset = load_dataset(CONFIG['hf_dataset'], CONFIG['dataset_name'])
    print(f"Veri seti yüklendi. Train örnek sayısı: {len(dataset['train'])}, Validation örnek sayısı: {len(dataset['validation'])}")
    
    tokenizer = build_tokenizer(dataset['train'])
    
    train_data = TextDataset(dataset['train'], tokenizer, CONFIG['seq_len'])
    valid_data = TextDataset(dataset['validation'], tokenizer, CONFIG['seq_len'])
    
    train_loader = DataLoader(train_data, batch_size=CONFIG['batch_size'], shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=CONFIG['batch_size'])
    
    print(f"Train loader örnek sayısı: {len(train_loader)}, Validation loader örnek sayısı: {len(valid_loader)}")
    return train_loader, valid_loader, tokenizer.get_vocab_size()

# Transformer Modeli
def build_transformer(src_vocab_size, tgt_vocab_size, src_seq_len, tgt_seq_len, d_model, N, h, dropout, d_ff):
    print("Transformer modeli oluşturuluyor...")
    # Gömme katmanları
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)
    print("Gömme katmanları oluşturuldu.")

    # Konumsal kodlama
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)
    print("Konumsal kodlama katmanları oluşturuldu.")

    # Encoder blokları
    encoder_blocks = []
    for i in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)
        print(f"Encoder block {i+1}/{N} oluşturuldu.")
    print("Encoder blokları tamamlandı.")

    # Decoder blokları
    decoder_blocks = []
    for i in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
        print(f"Decoder block {i+1}/{N} oluşturuldu.")
    print("Decoder blokları tamamlandı.")

    # Encoder ve Decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    print("Encoder ve Decoder oluşturuldu.")

    # Projeksiyon katmanı
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    print("Projeksiyon katmanı oluşturuldu.")

    # Transformer modeli
    transformer = Transformer(
        encoder=encoder,
        decoder=decoder,
        src_embed=src_embed,
        tgt_embed=tgt_embed,
        src_pos=src_pos,
        tgt_pos=tgt_pos,
        projection_layer=projection_layer
    )
    print("Transformer modeli başarıyla oluşturuldu.")

    # Parametreleri Xavier uniform ile başlat
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    print("Model parametreleri başlatıldı.")

    return transformer

# Eğitim Döngüsü
def train_model():
    print("Eğitim başlatılıyor...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Kullanılan cihaz: {device}")
    
    train_loader, valid_loader, vocab_size = get_data()
    
    transformer = build_transformer(
        src_vocab_size=vocab_size,
        tgt_vocab_size=vocab_size,
        src_seq_len=CONFIG['seq_len'],
        tgt_seq_len=CONFIG['seq_len'],
        d_model=CONFIG['d_model'],
        N=CONFIG['n_layers'],
        h=CONFIG['n_heads'],
        dropout=CONFIG['dropout'],
        d_ff=CONFIG['d_ff']
    ).to(device)
    
    optimizer = optim.Adam(transformer.parameters(), lr=CONFIG['lr'])
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    
    from torch.cuda.amp import GradScaler, autocast

# Eğitim döngüsü
def train_model():
    print("Eğitim başlatılıyor...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Kullanılan cihaz: {device}")
    
    train_loader, valid_loader, vocab_size = get_data()
    
    transformer = build_transformer(
        src_vocab_size=vocab_size,
        tgt_vocab_size=vocab_size,
        src_seq_len=CONFIG['seq_len'],
        tgt_seq_len=CONFIG['seq_len'],
        d_model=CONFIG['d_model'],
        N=CONFIG['n_layers'],
        h=CONFIG['n_heads'],
        dropout=CONFIG['dropout'],
        d_ff=CONFIG['d_ff']
    ).to(device)
    
    optimizer = optim.Adam(transformer.parameters(), lr=CONFIG['lr'])
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    scaler = GradScaler()
    accumulation_steps = 4  # Gradyan biriktirme adım sayısı

    for epoch in range(CONFIG['num_epochs']):
        transformer.train()
        total_loss = 0
        optimizer.zero_grad()
        
        for batch_idx, (src, tgt) in enumerate(train_loader):
            src = src.to(device)
            tgt = tgt.to(device)
            
            src_mask, tgt_mask = create_mask(src, tgt)
            
            with autocast():
                encoder_output = transformer.encode(src, src_mask)
                decoder_output = transformer.decode(encoder_output, src_mask, tgt, tgt_mask)
                output = transformer.project(decoder_output)
                loss = criterion(output.view(-1, vocab_size), tgt.view(-1)) / accumulation_steps
            
            scaler.scale(loss).backward()
            
            if (batch_idx + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            total_loss += loss.item() * accumulation_steps
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{CONFIG['num_epochs']}] Ortalama Loss: {avg_loss:.4f}")

# Modeli Çalıştırma
if __name__ == "__main__":
    train_model()

Eğitim başlatılıyor...
Kullanılan cihaz: cpu
Veri seti yükleniyor...
Veri seti yüklendi. Train örnek sayısı: 1801350, Validation örnek sayısı: 3760
Tokenizer oluşturuluyor...
Tokenizer eğitimi için batch 1 işleniyor...
Tokenizer eğitimi için batch 2 işleniyor...
Tokenizer eğitimi için batch 3 işleniyor...
Tokenizer eğitimi için batch 4 işleniyor...
Tokenizer eğitimi için batch 5 işleniyor...
Tokenizer eğitimi için batch 6 işleniyor...
Tokenizer eğitimi için batch 7 işleniyor...
Tokenizer eğitimi için batch 8 işleniyor...
Tokenizer eğitimi için batch 9 işleniyor...
Tokenizer eğitimi için batch 10 işleniyor...
Tokenizer eğitimi için batch 11 işleniyor...
Tokenizer eğitimi için batch 12 işleniyor...
Tokenizer eğitimi için batch 13 işleniyor...
Tokenizer eğitimi için batch 14 işleniyor...
Tokenizer eğitimi için batch 15 işleniyor...
Tokenizer eğitimi için batch 16 işleniyor...
Tokenizer eğitimi için batch 17 işleniyor...
Tokenizer eğitimi için batch 18 işleniyor...
Tokenizer eğitimi için b

C:\Users\emreq\AppData\Local\Temp\ipykernel_13168\2647060649.py:222: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\emreq\AppData\Local\Temp\ipykernel_13168\2647060649.py:236: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
